In [1]:
import sys
import os

  
print("Python Path:", sys.executable)

try:
    import cv2
    import mediapipe as mp
    print("all libraries are installed ✅")
except ImportError:
    print("there is an error")

Python Path: c:\Users\CLICK\OneDrive\Desktop\CV\yolo 2\venv\Scripts\python.exe
all libraries are installed ✅


In [ ]:
import cv2
import time
from ultralytics import YOLO
import mediapipe as mp

# download the model that i trained
model = YOLO('best (3).pt') 
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

cap = cv2.VideoCapture(0)
look_away_start = 0

# for full screen display
window_name = 'Absolute Gaze Detection'
cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    # activate YOLO detection on the current frame
    yolo_results = model(frame, conf=0.2, iou=0.5, verbose=False)
    for r in yolo_results:
        for box in r.boxes:
            label = model.names[int(box.cls[0])].lower()
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            # detect if the detected object is a phone, earphone, or paper and draw a red rectangle with a warning text
            if any(x in label for x in [ 'phone', 'earphone', 'paper']):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                cv2.putText(frame, f"!!! {label.upper()} !!!", (x1, y1-10), 1, 1, (0, 0, 255), 2)

    # Face Mesh for gaze detection 
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mesh_results = face_mesh.process(rgb_frame)
    gaze_direction = "Forward"

    if mesh_results.multi_face_landmarks:
        for face_landmarks in mesh_results.multi_face_landmarks:
            nose = face_landmarks.landmark[1]
            forehead = face_landmarks.landmark[10]
            chin = face_landmarks.landmark[152]
            
            #حسابات بؤبؤ العين اليسرى
            left_iris = face_landmarks.landmark[468]
            l_inner = face_landmarks.landmark[133]
            l_outer = face_landmarks.landmark[33]
            
            # النسبة الأفقية (h_ratio)
            h_ratio = (left_iris.x - l_outer.x) / (l_inner.x - l_outer.x)

            # حسابات الميل الرأسي (Head Pitch)
            face_height = chin.y - forehead.y
            nose_relative_pos = (nose.y - forehead.y) / face_height
 
            if h_ratio > 0.65 or nose.x > 0.65: 
                gaze_direction = "Looking Left"
            elif h_ratio < 0.35 or nose.x < 0.35: 
                gaze_direction = "Looking Right"
            elif nose_relative_pos < 0.42: 
                gaze_direction = "Looking Up"
            elif nose_relative_pos > 0.65: 
                gaze_direction = "Looking Down"

            # العرض على الشاشة
            color = (0, 255, 0) if gaze_direction == "Forward" else (0, 0, 255)
            cv2.putText(frame, f"GAZE: {gaze_direction}", (20, 50), 1, 2, color, 3)
            
            # منطق الـ Violation
            if gaze_direction != "Forward":
                if look_away_start == 0: look_away_start = time.time()
                if time.time() - look_away_start > 1.5:
                    cv2.putText(frame, "!!! GAZE VIOLATION !!!", (20, 110), 1, 2, (0,0,255), 3)
            else:
                look_away_start = 0

    #  للعرض علي الشاشه كامله 
    cv2.imshow(window_name, frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()